# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [77]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [78]:
# Load in file
weather_file = "filename.csv"

In [79]:
weather_file_df =pd.read_csv(weather_file)
weather_file_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atbasar,51.8000,68.3333,33.12,54,7,7.87,KZ,1650850018
1,1,ponta do sol,32.6667,-17.1000,65.01,87,100,10.87,PT,1650850269
2,2,kaitangata,-46.2817,169.8464,57.72,65,56,19.62,NZ,1650850269
3,3,hilo,19.7297,-155.0900,83.28,84,100,3.00,US,1650850270
4,4,barrow,71.2906,-156.7887,19.42,73,75,10.36,US,1650850272
...,...,...,...,...,...,...,...,...,...,...
562,562,wanning,18.8003,110.3967,84.85,75,28,14.67,CN,1650850714
563,563,hambantota,6.1241,81.1185,77.34,100,75,4.61,LK,1650850715
564,564,sola,-13.8833,167.5500,80.78,73,98,5.50,VU,1650850715
565,565,qujing,25.4833,103.7833,71.06,36,0,16.67,CN,1650850716


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [80]:
# Create a map using state centroid coordinates to set markers
marker_locations = weather_file_df[['Lat', 'Lng']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [81]:
# Fill NaN values and convert to float
humidity = weather_file_df["Humidity"].astype(float)

In [82]:
# BONUS
# Plost as a Hybrid map
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [83]:
#Fine tune the dataframe
Ideal_weather = weather_file_df.loc[(weather_file_df["Max Temp"] < 27) & (weather_file_df["Max Temp"] > 21) & 
                                    (weather_file_df["Wind Speed"] < 15)]
                                    
                                    

Ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,bilibino,68.0546,166.4372,22.42,75,100,4.32,RU,1650850284
54,54,kungurtug,50.5994,97.5228,21.11,47,76,7.09,RU,1650850309
92,92,ilulissat,69.2167,-51.1000,26.62,93,100,4.61,GL,1650850337
143,143,cherskiy,68.7500,161.3000,25.29,76,100,7.74,RU,1650850220
165,165,maniitsoq,65.4167,-52.9000,26.29,89,100,6.55,GL,1650850390
178,178,flin flon,54.7682,-101.8650,25.21,62,26,10.20,CA,1650850399
209,209,tuktoyaktuk,69.4541,-133.0374,26.60,80,0,5.75,CA,1650850424
216,216,keuruu,62.2667,24.7000,26.78,93,12,2.62,FI,1650850429
335,335,pangnirtung,66.1451,-65.7125,21.33,92,100,5.50,CA,1650850530
388,388,sisimiut,66.9395,-53.6735,23.77,92,72,1.14,GL,1650850572


In [84]:
Ideal_weather_df = Ideal_weather

In [86]:
Ideal_weather_df = Ideal_weather

In [87]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
Ideal_weather_df.to_csv("idealweather.csv", encoding="utf-8", index=False)

In [88]:
# Create a map using state centroid coordinates to set markers
Ideal_locations = Ideal_weather_df[['Lat', 'Lng']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(Ideal_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [89]:
weather_file_df['Hotel Name'] = ""
weather_file_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,atbasar,51.8000,68.3333,33.12,54,7,7.87,KZ,1650850018,
1,1,ponta do sol,32.6667,-17.1000,65.01,87,100,10.87,PT,1650850269,
2,2,kaitangata,-46.2817,169.8464,57.72,65,56,19.62,NZ,1650850269,
3,3,hilo,19.7297,-155.0900,83.28,84,100,3.00,US,1650850270,
4,4,barrow,71.2906,-156.7887,19.42,73,75,10.36,US,1650850272,


In [90]:
Hotel_df = weather_file_df
Hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,atbasar,51.8000,68.3333,33.12,54,7,7.87,KZ,1650850018,
1,1,ponta do sol,32.6667,-17.1000,65.01,87,100,10.87,PT,1650850269,
2,2,kaitangata,-46.2817,169.8464,57.72,65,56,19.62,NZ,1650850269,
3,3,hilo,19.7297,-155.0900,83.28,84,100,3.00,US,1650850270,
4,4,barrow,71.2906,-156.7887,19.42,73,75,10.36,US,1650850272,
...,...,...,...,...,...,...,...,...,...,...,...
562,562,wanning,18.8003,110.3967,84.85,75,28,14.67,CN,1650850714,
563,563,hambantota,6.1241,81.1185,77.34,100,75,4.61,LK,1650850715,
564,564,sola,-13.8833,167.5500,80.78,73,98,5.50,VU,1650850715,
565,565,qujing,25.4833,103.7833,71.06,36,0,16.67,CN,1650850716,


In [94]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = info_box_template.format(**row) 

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#geocoordinates
locations = Hotel_df[["Lat", "Lng"]]
target_type ="hotel"
target_radius = 5000


params = {
    "location": locations
    "radius": target_radius
    "type": target_type
    "key": g_key,
}

for index, row in Hotel_df.iterrows():
    
    locations = Hotel_df[["Lat", "Lng"]]

    print(f"Retrieving Results for Index {index}: {locations}.")
    response = requests.get(base_url, params=params).json()

    results = response['results']

    try:
        print(f"Nearby {locations} Hotel is {results[0]['Hotel Name']}.")
        
        Hotel_df.loc[index, 'Hotel Name'] = results[0]['Hotel Name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("------------")
        







SyntaxError: invalid syntax (Temp/ipykernel_8008/2299458843.py, line 24)

In [ ]:
# Add marker layer ontop of heat map


# Display figure
